<a href="https://colab.research.google.com/github/DIPANJAN001/Dipanjanpatra/blob/master/SolarManagementAgentfinal1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import numpy as np
import matplotlib.pyplot as plt
import os
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import random

In [4]:
import numpy as np
import gym
from gym.utils import seeding
from gym.spaces import Space, Discrete, MultiDiscrete,  Box
from gym.spaces.space import Space
import numpy as np
import random
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import metrics

In [47]:
class MyEnv3(gym.Env):
    
      def __init__(self):
        
        self.action_space = Discrete(4)
        self.observation_space=None
        #self.battery_efficiency=0.3
        self.battery_cap=30
        self.max_battery_discharge=15
        self.max_battery_charge=10
        self.max_battery_capacity=50
        #self.state=[]
        #self.reward, self.done, self.info= None, None, None
        #self.grid_import=0
        #self.grid_export=0
        self.info=None
      def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

      def _step(self,action):
        state=[]
        grid_import_=0
        grid_export_=0
        done=False
        load=np.random.randint(100,150)
        pv=np.random.randint(80,130)
        battery_state=self.battery_cap
        #net_load=max((load-power),0)
        #net_rem=max(net_load-(self.battery_efficiency*self.battery_cap),0)
        info={}
        reward=0
        #battery_chng=0
        
        if action== 0:
          grid_export_= max(pv-load-self.max_battery_charge,0) # grid export
          self.battery_cap= max(min(self.battery_cap+min(self.max_battery_charge,pv-load),50),0)
          grid_import_=0
          info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
          }
        if action== 1:
          grid_export_= max(pv-load-self.max_battery_charge,0) # grid export
          self.battery_cap= max(min(self.battery_cap+min(self.max_battery_charge,pv-load),50),0)
          grid_import_=0
          # only battery charge
          info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
          }
        if action== 2:
            grid_export_=0
            self.battery_cap= min(max(self.battery_cap-self.max_battery_discharge,0),50)# battery discharge
            grid_import_=max(load-pv-self.max_battery_discharge,0)
            info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
             }
        if action== 3:
          grid_export_=0
          self.battery_cap= min(max(self.battery_cap-min(self.max_battery_discharge,load-pv),0),50)#change in battery
          grid_import_=max(load-pv-self.max_battery_discharge,0)
         #only battery discharge
          info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
                    }     
        
        reward=grid_export_*8-grid_import_*10
        
        if(self.battery_cap<7.0):
              done=True
        state.append([battery_state,load,pv,action,reward])
        #state.append(load)
        #state.append(pv)
        return state, reward,self.battery_cap, done,info
        
      def _reset(self):
        self.battery_cap=30
        #self.done=False
        return self.battery_cap

In [48]:
env=MyEnv3()

In [7]:
'''class Agent:
    def __init__(self,state_size,action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95 #Discount Factor
        self.epsilon = 1.0 # Exploration Rate: How much to act randomly, 
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.learning_rate = 0.001 
        self.model = self._create_model()
        
    
    def _create_model(self):
        #Neural Network To Approximate Q-Value function
        model = Sequential()
        model.add(Dense(24,input_dim=self.state_size,activation='relu')) #1st Hidden Layer
        model.add(Dense(24,activation='relu')) #2nd Hidden Layer
        model.add(Dense(self.action_size,activation='linear'))
        model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))
        return model
    
    def remember(self,state,action,reward,next_state,done):
        self.memory.append((state,action,reward,next_state,done)) #remembering previous experiences
        
    def act(self,state):
        # Exploration vs Exploitation
        if np.random.rand()<=self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state) # predict reward value based upon current state
        return np.argmax(act_values[0]) #Left or Right
    
    def train(self,batch_size=32): #method that trains NN with experiences sampled from memory
        minibatch = random.sample(self.memory,batch_size)
        for state,action,reward,next_state,done in minibatch:
            
            if not done: #boolean 
                target = reward + self.gamma*np.amax(self.model.predict(next_state)[0])
            else:
                target = reward
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state,target_f,epochs=1,verbose=0) #single epoch, x =state, y = target_f, loss--> target_f - 
            
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def load(self,name):
        self.model.load_weights(name)
    def save(self,name):
        self.model.save_weights(name)'''

In [8]:
agent = Agent(state_size=3,action_size=4)
done = False
state_size = 3
action_size =4
batch_size = 32

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
'''agent = Agent(state_size, action_size) # initialise agent
done = False
for e in range(n_episodes):
    state = env.reset()
    state = np.reshape(state,[1,state_size])
    
    for time in range(5000):
        env.render()
        action = agent.act(state) #action is 0 or 1
        next_state,reward,done,other_info = env.step(action) 
        reward = reward if not done else -10
        next_state = np.reshape(next_state,[1,state_size])
        agent.remember(state,action,reward,next_state,done)
        state = next_state
        
        if done:
            print("Game Episode :{}/{}, High Score:{},Exploration Rate:{:.2}".format(e,n_episodes,time,agent.epsilon))
            break
            
    if len(agent.memory)>batch_size:
        agent.train(batch_size)
    
    if e%50==0:
        agent.save(output_dir+"weights_"+'{:04d}'.format(e)+".hdf5")
        
env.close()'''

In [51]:
for e in range(400): #Episode
    #Play 20 episodes 
    score = 0
    observation = env._reset()
    #print(observation)
    done=False
    while not done:
        #env.render()
        action = env.action_space.sample()
        #print(action)
        observation1,reward,observation4,done,info = env._step(action)
        print(list(observation1))
        print(observation4)
        score+=round(reward)
        #print(info['load'],info['power'])
      
    print('episode:{} Profit:{}'.format(e,score))

[[30, 143, 110, 1, 0]]
0
episode:0 Profit:0
[[30, 119, 124, 0, 0]]
35
[[35, 141, 102, 0, 0]]
0
episode:1 Profit:0
[[30, 129, 100, 3, -140]]
15
[[15, 114, 83, 3, -160]]
0
episode:2 Profit:-300
[[30, 109, 92, 2, -20]]
15
[[15, 104, 92, 0, 0]]
3
episode:3 Profit:-20
[[30, 104, 92, 2, 0]]
15
[[15, 118, 102, 3, -10]]
0
episode:4 Profit:-10
[[30, 120, 123, 3, 0]]
33
[[33, 139, 113, 1, 0]]
7
[[7, 105, 91, 1, 0]]
0
episode:5 Profit:0
[[30, 110, 109, 1, 0]]
29
[[29, 126, 80, 3, -310]]
14
[[14, 116, 108, 3, 0]]
6
episode:6 Profit:-310
[[30, 137, 100, 3, -220]]
15
[[15, 149, 95, 3, -390]]
0
episode:7 Profit:-610
[[30, 122, 98, 1, 0]]
6
episode:8 Profit:0
[[30, 105, 90, 2, 0]]
15
[[15, 141, 123, 2, -30]]
0
episode:9 Profit:-30
[[30, 144, 93, 2, -360]]
15
[[15, 114, 113, 1, 0]]
14
[[14, 142, 106, 2, -210]]
0
episode:10 Profit:-570
[[30, 116, 101, 0, 0]]
15
[[15, 100, 83, 2, -20]]
0
episode:11 Profit:-20
[[30, 141, 115, 1, 0]]
4
episode:12 Profit:0
[[30, 112, 96, 0, 0]]
14
[[14, 127, 90, 1, 0]]
0
ep